# Compare kernel statistics

This notebook compares kernel statistics between two Nsight Systems SQLite report files.

We'll specifically compare the `set_prognostic_edmf_precomputed_quantities_precipitation` kernel between:
- `baseline.sqlite` - baseline run
- `mod.sqlite` - modified run

## Imports and setup

In [1]:
import difflib
import re
import sqlite3
import unicodedata
from pathlib import Path

import pandas as pd
from IPython.display import display

## Helper functions

In [2]:
def get_kernel_stats(db_path, kernel_name_pattern):
    """
    Extract kernel statistics from an nsys SQLite database.

    Args:
        db_path: Path to the SQLite database file
        kernel_name_pattern: SQL LIKE pattern to match kernel names

    Returns:
        Dictionary with aggregated statistics and list of all kernel invocations
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Query to get all kernel invocations with the specified name
    query = """
    SELECT
        s.value as kernelName,
        k.start,
        k.end,
        (k.end - k.start) as duration_ns,
        k.gridX,
        k.gridY,
        k.gridZ,
        k.blockX,
        k.blockY,
        k.blockZ,
        k.registersPerThread,
        k.staticSharedMemory,
        k.dynamicSharedMemory,
        k.localMemoryPerThread,
        k.localMemoryTotal,
        k.deviceId,
        k.streamId,
        k.launchType,
        k.sharedMemoryExecuted,
        k.correlationId,
        k.globalPid
    FROM CUPTI_ACTIVITY_KIND_KERNEL k
    JOIN StringIds s ON k.demangledName = s.id
    WHERE s.value LIKE ?
    ORDER BY k.start
    """

    cursor.execute(query, (kernel_name_pattern,))
    rows = cursor.fetchall()

    if not rows:
        conn.close()
        return None, None

    # Calculate statistics
    durations = [row[3] for row in rows]

    stats = {
        "kernel_name": rows[0][0],
        "invocation_count": len(rows),
        "duration_ns": {
            "total": sum(durations),
            "mean": sum(durations) / len(durations),
            "min": min(durations),
            "max": max(durations),
            "median": sorted(durations)[len(durations) // 2],
        },
        "grid_dims": {
            "x": rows[0][4],
            "y": rows[0][5],
            "z": rows[0][6],
        },
        "block_dims": {
            "x": rows[0][7],
            "y": rows[0][8],
            "z": rows[0][9],
        },
        "registers_per_thread": rows[0][10],
        "static_shared_memory": rows[0][11],
        "dynamic_shared_memory": rows[0][12],
        "local_memory_per_thread": rows[0][13],
        "local_memory_total": rows[0][14],
        "device_id": rows[0][15],
        "stream_id": rows[0][16],
        "launch_type": rows[0][17],
        "shared_memory_executed": rows[0][18],
        "correlation_id": rows[0][19],
        "global_pid": rows[0][20],
    }

    # Calculate theoretical occupancy
    # Occupancy = (active warps per SM) / (max warps per SM) * 100%
    block_size = rows[0][7] * rows[0][8] * rows[0][9]
    warps_per_block = (block_size + 31) // 32
    max_warps_per_sm = 48

    # Register file size: ~49KB per SM (determined by occupancy calculator matching nsys)
    max_registers_per_sm = 49152  # registers available per SM
    registers_per_block = rows[0][10] * block_size
    blocks_limited_by_registers = (
        max(1, max_registers_per_sm // registers_per_block)
        if registers_per_block > 0
        else 8
    )

    # Shared memory limit (if applicable, but usually not limiting for this kernel)
    max_shared_mem_per_sm = 96000
    shared_mem_per_block = rows[0][11] + rows[0][12]
    blocks_limited_by_shared_mem = (
        max(1, max_shared_mem_per_sm // shared_mem_per_block)
        if shared_mem_per_block > 0
        else 8
    )

    # Effective blocks per SM is the minimum of register and shared memory constraints
    blocks_per_sm = min(
        blocks_limited_by_registers, blocks_limited_by_shared_mem
    )

    # Calculate occupancy
    active_warps = warps_per_block * blocks_per_sm
    occupancy = (active_warps / max_warps_per_sm) * 100
    stats["theoretical_occupancy"] = occupancy
    conn.close()
    return stats, rows


def get_launch_type_name(launch_type_id):
    """Convert launch type ID to human-readable name."""
    launch_types = {
        0: "Regular",
        1: "Cooperative",
    }
    return launch_types.get(launch_type_id, f"Unknown ({launch_type_id})")


In [3]:
def format_duration(ns):
    """Convert nanoseconds to human-readable format."""
    if ns < 1000:
        return f"{ns:.2f} ns"
    elif ns < 1_000_000:
        return f"{ns / 1000:.2f} µs"
    elif ns < 1_000_000_000:
        return f"{ns / 1_000_000:.2f} ms"
    else:
        return f"{ns / 1_000_000_000:.2f} s"

In [4]:
def compare_stats(baseline_stats, mod_stats):
    """
    Compare two sets of kernel statistics and display results in a single table.

    Args:
        baseline_stats: Statistics from baseline run
        mod_stats: Statistics from modified run
    """
    if baseline_stats is None or mod_stats is None:
        print(
            "Error: Could not retrieve statistics from one or both databases"
        )
        return

    print("\n" + "=" * 140)
    print(f"KERNEL: {baseline_stats['kernel_name']}")
    print("=" * 140 + "\n")

    # Build comprehensive data structure with all metrics
    data = {"Metric": [], "Baseline": [], "Modified": [], "% Change": []}

    # Timing Statistics
    metrics = [
        (
            "Invocations",
            baseline_stats["invocation_count"],
            mod_stats["invocation_count"],
            True,
        ),
        (
            "Total Duration",
            baseline_stats["duration_ns"]["total"],
            mod_stats["duration_ns"]["total"],
            False,
        ),
        (
            "Mean Duration",
            baseline_stats["duration_ns"]["mean"],
            mod_stats["duration_ns"]["mean"],
            False,
        ),
        (
            "Median Duration",
            baseline_stats["duration_ns"]["median"],
            mod_stats["duration_ns"]["median"],
            False,
        ),
        (
            "Min Duration",
            baseline_stats["duration_ns"]["min"],
            mod_stats["duration_ns"]["min"],
            False,
        ),
        (
            "Max Duration",
            baseline_stats["duration_ns"]["max"],
            mod_stats["duration_ns"]["max"],
            False,
        ),
    ]

    for metric_name, baseline_val, mod_val, is_count in metrics:
        data["Metric"].append(metric_name)

        if is_count:
            data["Baseline"].append(f"{baseline_val:,}")
            data["Modified"].append(f"{mod_val:,}")
            pct = (
                (mod_val - baseline_val) / baseline_val * 100
                if baseline_val > 0
                else 0
            )
        else:
            data["Baseline"].append(format_duration(baseline_val))
            data["Modified"].append(format_duration(mod_val))
            pct = (mod_val - baseline_val) / baseline_val * 100

        sign = "+" if pct > 0 else ""
        data["% Change"].append(f"{sign}{pct:.2f}%")

    # Kernel Dimensions & Launch
    data["Metric"].append("Grid Dimensions")
    data["Baseline"].append(
        f"<{baseline_stats['grid_dims']['x']}, {baseline_stats['grid_dims']['y']}, {baseline_stats['grid_dims']['z']}>"
    )
    data["Modified"].append(
        f"<{mod_stats['grid_dims']['x']}, {mod_stats['grid_dims']['y']}, {mod_stats['grid_dims']['z']}>"
    )
    data["% Change"].append("—")

    data["Metric"].append("Block Dimensions")
    data["Baseline"].append(
        f"<{baseline_stats['block_dims']['x']}, {baseline_stats['block_dims']['y']}, {baseline_stats['block_dims']['z']}>"
    )
    data["Modified"].append(
        f"<{mod_stats['block_dims']['x']}, {mod_stats['block_dims']['y']}, {mod_stats['block_dims']['z']}>"
    )
    data["% Change"].append("—")

    block_size_baseline = (
        baseline_stats["block_dims"]["x"]
        * baseline_stats["block_dims"]["y"]
        * baseline_stats["block_dims"]["z"]
    )
    block_size_mod = (
        mod_stats["block_dims"]["x"]
        * mod_stats["block_dims"]["y"]
        * mod_stats["block_dims"]["z"]
    )
    data["Metric"].append("Block Size (threads)")
    data["Baseline"].append(f"{block_size_baseline:,}")
    data["Modified"].append(f"{block_size_mod:,}")
    data["% Change"].append("—")

    grid_size_baseline = (
        baseline_stats["grid_dims"]["x"]
        * baseline_stats["grid_dims"]["y"]
        * baseline_stats["grid_dims"]["z"]
    )
    grid_size_mod = (
        mod_stats["grid_dims"]["x"]
        * mod_stats["grid_dims"]["y"]
        * mod_stats["grid_dims"]["z"]
    )
    data["Metric"].append("Grid Size (blocks)")
    data["Baseline"].append(f"{grid_size_baseline:,}")
    data["Modified"].append(f"{grid_size_mod:,}")
    data["% Change"].append("—")

    total_threads_baseline = grid_size_baseline * block_size_baseline
    total_threads_mod = grid_size_mod * block_size_mod
    data["Metric"].append("Total Threads")
    data["Baseline"].append(f"{total_threads_baseline:,}")
    data["Modified"].append(f"{total_threads_mod:,}")
    data["% Change"].append("—")

    data["Metric"].append("Launch Type")
    data["Baseline"].append(
        get_launch_type_name(baseline_stats["launch_type"])
    )
    data["Modified"].append(get_launch_type_name(mod_stats["launch_type"]))
    data["% Change"].append("—")

    # Register Usage
    data["Metric"].append("Registers Per Thread")
    data["Baseline"].append(f"{baseline_stats['registers_per_thread']}")
    data["Modified"].append(f"{mod_stats['registers_per_thread']}")
    if (
        baseline_stats["registers_per_thread"]
        != mod_stats["registers_per_thread"]
    ):
        pct = (
            (
                mod_stats["registers_per_thread"]
                - baseline_stats["registers_per_thread"]
            )
            / baseline_stats["registers_per_thread"]
            * 100
        )
        sign = "+" if pct > 0 else ""
        data["% Change"].append(f"{sign}{pct:.2f}%")
    else:
        data["% Change"].append("—")

    # Memory Usage
    memory_metrics = [
        (
            "Static Shared Memory (bytes)",
            baseline_stats["static_shared_memory"],
            mod_stats["static_shared_memory"],
        ),
        (
            "Dynamic Shared Memory (bytes)",
            baseline_stats["dynamic_shared_memory"],
            mod_stats["dynamic_shared_memory"],
        ),
        (
            "Shared Memory Executed (bytes)",
            baseline_stats["shared_memory_executed"],
            mod_stats["shared_memory_executed"],
        ),
        (
            "Local Memory Per Thread (bytes)",
            baseline_stats["local_memory_per_thread"],
            mod_stats["local_memory_per_thread"],
        ),
        (
            "Local Memory Total (bytes)",
            baseline_stats["local_memory_total"],
            mod_stats["local_memory_total"],
        ),
    ]

    for metric_name, baseline_val, mod_val in memory_metrics:
        data["Metric"].append(metric_name)
        data["Baseline"].append(f"{baseline_val:,}")
        data["Modified"].append(f"{mod_val:,}")

        if baseline_val == mod_val:
            data["% Change"].append("—")
        elif baseline_val == 0:
            data["% Change"].append("—")
        else:
            pct = (mod_val - baseline_val) / baseline_val * 100
            sign = "+" if pct > 0 else ""
            data["% Change"].append(f"{sign}{pct:.2f}%")

    # Theoretical Occupancy
    data["Metric"].append("Theoretical Occupancy (%)")
    data["Baseline"].append(f"{baseline_stats['theoretical_occupancy']:.1f}%")
    data["Modified"].append(f"{mod_stats['theoretical_occupancy']:.1f}%")
    data["% Change"].append("—")

    # Create and display the combined dataframe
    df = pd.DataFrame(data)
    display(df)

    # Performance summary
    print("\n" + "=" * 140)
    baseline_mean = baseline_stats["duration_ns"]["mean"]
    mod_mean = mod_stats["duration_ns"]["mean"]

    if mod_mean < baseline_mean:
        improvement = ((baseline_mean - mod_mean) / baseline_mean) * 100
        print(
            f"✅ Modified version is {improvement:.2f}% FASTER (mean duration)"
        )
    elif mod_mean > baseline_mean:
        regression = ((mod_mean - baseline_mean) / baseline_mean) * 100
        print(
            f"⚠️  Modified version is {regression:.2f}% SLOWER (mean duration)"
        )
    else:
        print("➡️  No change in mean duration")

    print("=" * 140)


## Configuration

In [5]:
# Database paths
baseline_db = Path("../results/nsys/baseline.sqlite")
mod_db = Path("../results/nsys/mod.sqlite")

# Kernel name pattern to match (avoid fragile line numbers)
kernel_pattern = "%set_prognostic_edmf_precomputed_quantities_precipitation%"

# Check if files exist
if not baseline_db.exists():
    print(f"Error: Baseline database not found: {baseline_db}")
else:
    print(f"✓ Found baseline database: {baseline_db}")

if not mod_db.exists():
    print(f"Error: Modified database not found: {mod_db}")
else:
    print(f"✓ Found modified database: {mod_db}")

✓ Found baseline database: ../results/nsys/baseline.sqlite
✓ Found modified database: ../results/nsys/mod.sqlite


## Compare kernel statistics

In [6]:
def normalize_kernel_name(name):
    """Normalize kernel names for robust matching."""
    normalized = unicodedata.normalize("NFKC", name)
    normalized = "".join(ch for ch in normalized if ch.isprintable())
    normalized = re.sub(r"\s+", " ", normalized).strip()
    return normalized


def strip_mod_suffix(name):
    """Remove a loose mod suffix for matching."""
    return re.sub(r"(?:_|\s)mod\b", "", name).strip()


def list_available_kernels(db_path, pattern="%set_prognostic_edmf%"):
    """
    List all kernels matching a pattern in the database.

    Args:
        db_path: Path to SQLite database
        pattern: SQL LIKE pattern for kernel names
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    query = """
    SELECT DISTINCT s.value as kernelName, COUNT(*) as count
    FROM CUPTI_ACTIVITY_KIND_KERNEL k
    JOIN StringIds s ON k.demangledName = s.id
    WHERE s.value LIKE ?
    GROUP BY s.value
    ORDER BY s.value
    """

    cursor.execute(query, (pattern,))
    results = cursor.fetchall()
    conn.close()

    return results


def list_top_kernels(db_path, pattern="%", limit=15):
    """
    List top kernels by total duration, filtered by a name pattern.

    Args:
        db_path: Path to SQLite database
        pattern: SQL LIKE pattern for kernel names
        limit: Number of kernels to return
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    query = """
    SELECT s.value as kernelName, SUM(k.end - k.start) as total_ns, COUNT(*) as count
    FROM CUPTI_ACTIVITY_KIND_KERNEL k
    JOIN StringIds s ON k.demangledName = s.id
    WHERE s.value LIKE ?
    GROUP BY s.value
    ORDER BY total_ns DESC
    LIMIT ?
    """

    cursor.execute(query, (pattern, limit))
    results = cursor.fetchall()
    conn.close()

    return results


def select_kernel_name(baseline_top, mod_top):
    """
    Select the most expensive baseline kernel and the closest name match in modified.
    """
    if not baseline_top:
        raise RuntimeError("No kernels matched in baseline")

    if not mod_top:
        raise RuntimeError("No kernels matched in modified")

    baseline_kernel, baseline_total, _ = baseline_top[0]
    baseline_norm = strip_mod_suffix(normalize_kernel_name(baseline_kernel))

    print(
        f"Selected baseline kernel: {baseline_kernel} (total={format_duration(baseline_total)})"
    )

    best = None
    for mod_kernel, mod_total, _ in mod_top:
        mod_norm = strip_mod_suffix(normalize_kernel_name(mod_kernel))
        score = difflib.SequenceMatcher(None, baseline_norm, mod_norm).ratio()
        candidate = (score, mod_total, mod_kernel)
        if best is None or candidate > best:
            best = candidate

    best_score, best_total, best_kernel = best
    print(
        f"Selected modified kernel: {best_kernel} (total={format_duration(best_total)}, score={best_score:.3f})"
    )

    return baseline_kernel, best_kernel


# List available kernels
print("Available kernels in baseline database:")
baseline_kernels = list_available_kernels(baseline_db, kernel_pattern)
for kernel_name, count in baseline_kernels:
    print(f"  [{count:4d} invocations] {kernel_name}")

print("\nAvailable kernels in modified database:")
mod_kernels = list_available_kernels(mod_db, kernel_pattern)
for kernel_name, count in mod_kernels:
    print(f"  [{count:4d} invocations] {kernel_name}")

print("\nTop kernels in baseline (by total time):")
baseline_top = list_top_kernels(baseline_db, kernel_pattern, limit=15)
for kernel_name, total_ns, count in baseline_top:
    print(
        f"  {format_duration(total_ns):>10}  [{count:4d} invocations] {kernel_name}"
    )

print("\nTop kernels in modified (by total time):")
mod_top = list_top_kernels(mod_db, kernel_pattern, limit=15)
for kernel_name, total_ns, count in mod_top:
    print(
        f"  {format_duration(total_ns):>10}  [{count:4d} invocations] {kernel_name}"
    )

baseline_kernel_name, mod_kernel_name = select_kernel_name(
    baseline_top, mod_top
)

Available kernels in baseline database:
  [  25 invocations] set_prognostic_edmf_precomputed_quantities_precipitation__FILE_ClimaAtmos_jl_src_cache_prognostic_edmf_precomputed_quantities_jl_L532
  [  25 invocations] set_prognostic_edmf_precomputed_quantities_precipitation__FILE_ClimaAtmos_jl_src_cache_prognostic_edmf_precomputed_quantities_jl_L538
  [  25 invocations] set_prognostic_edmf_precomputed_quantities_precipitation__FILE_ClimaAtmos_jl_src_cache_prognostic_edmf_precomputed_quantities_jl_L545
  [  25 invocations] set_prognostic_edmf_precomputed_quantities_precipitation__FILE_ClimaAtmos_jl_src_cache_prognostic_edmf_precomputed_quantities_jl_L551
  [  25 invocations] set_prognostic_edmf_precomputed_quantities_precipitation__FILE_ClimaAtmos_jl_src_cache_prognostic_edmf_precomputed_quantities_jl_L593
  [  25 invocations] set_prognostic_edmf_precomputed_quantities_precipitation__FILE_ClimaAtmos_jl_src_cache_prognostic_edmf_precomputed_quantities_jl_L612
  [ 460 invocations] set_progn

  [  25 invocations] set_prognostic_edmf_precomputed_quantities_precipitation__FILE_ClimaAtmos_jl_mod_src_cache_prognostic_edmf_precomputed_quantities_jl_L541
  [  25 invocations] set_prognostic_edmf_precomputed_quantities_precipitation__FILE_ClimaAtmos_jl_mod_src_cache_prognostic_edmf_precomputed_quantities_jl_L547
  [  25 invocations] set_prognostic_edmf_precomputed_quantities_precipitation__FILE_ClimaAtmos_jl_mod_src_cache_prognostic_edmf_precomputed_quantities_jl_L554
  [  25 invocations] set_prognostic_edmf_precomputed_quantities_precipitation__FILE_ClimaAtmos_jl_mod_src_cache_prognostic_edmf_precomputed_quantities_jl_L560
  [  25 invocations] set_prognostic_edmf_precomputed_quantities_precipitation__FILE_ClimaAtmos_jl_mod_src_cache_prognostic_edmf_precomputed_quantities_jl_L602
  [  25 invocations] set_prognostic_edmf_precomputed_quantities_precipitation__FILE_ClimaAtmos_jl_mod_src_cache_prognostic_edmf_precomputed_quantities_jl_L621
  [ 460 invocations] set_prognostic_edmf_preco

In [7]:
# Get statistics from both databases
if baseline_kernel_name is None or mod_kernel_name is None:
    print("Error: Missing kernel name selection; check the listing above.")
else:
    baseline_stats, baseline_rows = get_kernel_stats(
        baseline_db, baseline_kernel_name
    )
    mod_stats, mod_rows = get_kernel_stats(mod_db, mod_kernel_name)

    # Compare and display results
    compare_stats(baseline_stats, mod_stats)


KERNEL: set_prognostic_edmf_precomputed_quantities_precipitation__FILE_ClimaAtmos_jl_src_cache_prognostic_edmf_precomputed_quantities_jl_L593



,Metric,Baseline,Modified,% Change
0,Invocations,25,460,+1740.00%
1,Total Duration,461.52 ms,9.76 ms,-97.88%
2,Mean Duration,18.46 ms,21.22 µs,-99.89%
3,Median Duration,18.39 ms,20.83 µs,-99.89%
4,Min Duration,18.37 ms,20.45 µs,-99.89%
5,Max Duration,20.09 ms,26.46 µs,-99.87%
6,Grid Dimensions,"<4, 4, 1536>","<4, 4, 1536>",—
7,Block Dimensions,"<64, 1, 1>","<64, 1, 1>",—
8,Block Size (threads),64,64,—
9,Grid Size (blocks),"24,576","24,576",—



✅ Modified version is 99.89% FASTER (mean duration)
